In [1]:
import pyro
import pyro.distributions as dist
from pyro.infer import Importance, EmpiricalMarginal
from statistics import mean
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
pyro.set_rng_seed(101)

In [2]:
df = pd.read_csv("data/test.csv")

```r
dag1 <- model2network("[Neigh][BPB][Zest|BPB:Neigh][Rent|BPB:Neigh][ROI|Zest:Rent]")
dag2 <- model2network("[latitude][longitude][BPB][Zest|BPB:latitude:longitude][Rent|BPB:latitude:longitude][ROI|Zest:Rent]")
dag3 <- model2network("[latlng][Type][BPB|Type][Zest|BPB:latlng:Type][Rent|BPB:latlng][ROI|Zest:Rent]")
dag4 <- model2network("[latlng][BPB][Zest|BPB:latlng][Rent|BPB:latlng][ROI|Zest:Rent]")
dag5 <- model2network("[Neigh][BPB][Zest|BPB:Neigh][Rent|BPB][ROI|Zest:Rent]")
```

# Building Models and CPTs for Nodes

Taken from [this](https://pythonfordatascience.org/anova-2-way-n-way/) tutorial

- Grab standard error and use as Nrent

In [3]:
rent_model = ols('Rent ~ C(BPB)', df).fit()

# Seeing if the overall model is significant
print(f"Overall model F({rent_model.df_model: .0f},{rent_model.df_resid: .0f}) = {rent_model.fvalue: .3f}, p = {rent_model.f_pvalue: .4f}")


Overall model F( 4, 200) =  13.359, p =  0.0000


In [4]:
rent_model.predict(pd.DataFrame({'BPB':[4]}))

0    196.4375
dtype: float64

In [5]:
res = sm.stats.anova_lm(rent_model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(BPB),242221.823818,4.0,13.359283,1.148343e-09
Residual,906567.424962,200.0,NaN,NaN


- Do OLS model for Zestimates
- Get Neighborhood Marginal
- Get BPB marginal
- Get ROI function in the code

In [20]:
zest_model = ols('Zest ~ C(BPB) + C(Neigh)', df).fit()

# Seeing if the overall model is significant
print(f"Overall model F({zest_model.df_model: .0f},{zest_model.df_resid: .0f}) = {zest_model.fvalue: .3f}, p = {zest_model.f_pvalue: .4f}")


Overall model F( 5, 199) =  8.702, p =  0.0000


In [12]:
res = sm.stats.anova_lm(zest_model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(BPB),9.832538e+11,4.0,0.412598,7.994449e-01
C(Neigh),2.399361e+13,1.0,40.273283,1.512155e-09
C(BPB):C(Neigh),8.653439e+11,4.0,0.363120,8.346705e-01
Residual,1.161751e+14,195.0,NaN,NaN


In [24]:
zest_model.predict(pd.DataFrame({'BPB':[2], 'Neigh':['Neigh_South']}))

0    1.063638e+06
dtype: float64

In [7]:
# zest_lr_dict = {}
# for bpb in df.BPB.unique():
#     subset = df[df.BPB == bpb]
#     lr = LinearRegression()
#     lr.fit(subset[['latlng']], subset.Zest)
#     zest_lr_dict[bpb] = lr
#     print(r2_score(subset.Zest, lr.predict(subset[['latlng']])), subset.shape[0])
    
# print("\n\n")
# rent_lr_dict = {}

# for bpb in df.BPB.unique():
#     subset = df[df.BPB == bpb]
#     lr = LinearRegression()
#     lr.fit(subset[['latlng']], subset.Rent)
#     rent_lr_dict[bpb] = lr
#     print(r2_score(subset.Rent, lr.predict(subset[['latlng']])), subset.shape[0])

In [25]:
float(len(df.loc[df['Neigh'] == 'Neigh_North']))/len(df)

0.5756097560975609

In [29]:
[(df.BPB == a).sum()/len(df) for a in [2, 3, 4, 5, 6]]

[0.13658536585365855,
 0.16097560975609757,
 0.3902439024390244,
 0.25853658536585367,
 0.05365853658536585]

In [33]:
def calculate_monthly(P, mortgage_rate, num_years):
    n = num_years * 12
    monthly_i = mortgage_rate / 12
    numerator = monthly_i * (1 + monthly_i) ** n
    denominator = ((1 + monthly_i) ** n) - 1
    return P * numerator / denominator


def airbnb_income(price, inflation_rate, num_years):
    total = 0
    for year_number in range(num_years):
        curr_inflation = (1 + inflation_rate) ** year_number
        total += (price * curr_inflation) * 12
    return total

def roi(zestimate, inflation_rate, mortgage_rate, num_years, rental_price, down_payment_percent):
    down_payment = zestimate * down_payment_percent
    P = zestimate * (1 - down_payment_percent)

    incurred_cost = calculate_monthly(P, mortgage_rate, num_years) * 12 * num_years + down_payment

    income = airbnb_income(price=rental_price,
                           inflation_rate=inflation_rate,
                           num_years=num_years)

    return (income - incurred_cost) / incurred_cost


INFLATION_RATE = 0.028
MORTGAGE_RATE = 0.036
NUM_YEARS = 30
DOWN_PAYMENT = 0.20


In [32]:
Neigh_alias = ['Neigh_North','Neigh_South']
BPB_alias = ['2', '3', '4', '5', '6']

north_prob = float(len(df.loc[df['Neigh'] == 'Neigh_North']))/len(df)
bpb_prob = [(df.BPB == a).count()/len(df) for a in BPB_alias]
        

Neigh_prob = torch.tensor([north_prob, 1- north_prob])
BPB_prob = torch.tensor(bpb_prob)

def model():
    Neigh = pyro.sample("Neigh", dist.Categorical(probs=Neigh_prob))
    BPB = pyro.sample("BPB", dist.Categorical(probs=BPB_prob))
    Rent = pyro.sample("Rent", dist.Delta(rent_model.predict(pd.DataFrame({'BPB':[BPB]}))))
    Zest = pyro.sample("Zest", dist.Delta(zest_model.predict(pd.DataFrame({'BPB':[BPB], 'Neigh':[Neigh]}))))
    ROI = pyro.sample("ROI", dist.Delta(roi(zestimate=Zest,
        rental_price=Rent,
        inflation_rate=INFLATION_RATE,
        mortgage_rate=MORTGAGE_RATE,
        num_years=NUM_YEARS,
        down_payment_percent=DOWN_PAYMENT)))
    return {'Neigh':Neigh, 'BPB':BPB, 'Rent':Rent, 'Zest': Zest, 'ROI':ROI}
print(model())

PatsyError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error converting data to categorical: observation with value tensor(1) does not match any of the expected levels (expected: [2, 3, ..., 5, 6])
    Rent ~ C(BPB)
           ^^^^^^

In [ ]:
exogenous_dists = {
    'Nlatlng': dist.Bernoulli(torch.tensor(.5)),
    'Ny': dist.Bernoulli(torch.tensor(.5)),
    'Nz': dist.Bernoulli(torch.tensor(.5))
}

zest_lr = LinearRegression()
zest_lr.fit(df[['BPB', 'latlng']], df.Zest)


rent_lr = LinearRegression()
rent_lr.fit(df[['BPB', 'latlng']], df.Rent)

def model(exogenous_dists):
    Nx = pyro.sample('Nlatlng', exogenous_dists['Nlatlng'])
    Ny = pyro.sample('Ny', exogenous_dists['Ny'])
    Nz = pyro.sample('Nz', exogenous_dists['Nz'])
    
    X = pyro.sample('X', dist.Delta(Nx))
    Y = pyro.sample('Y', dist.Delta(Ny))
    
    Nz = dist.Normal(sd_zest_lr)
    
    Zest = pyro.sample('Zest', dist.Delta( + Nz))
    
    or_part = min(X+Y, torch.tensor(1))
    and_part = (X * Y)
    
    Z = pyro.sample('Z', dist.Delta(Nz * or_part + (1 - Nz) * and_part))
    return X, Y, Z